In [1]:
import requests 
import bs4

print(requests.__version__)
print(bs4.__version__)

2.31.0
4.12.3


## 네이버 상태 코드

In [2]:
URL = 'https://www.naver.com/'

req = requests.get(URL)
print(req.status_code)

200


In [4]:
#req.text

In [22]:
from bs4 import BeautifulSoup

import pandas as pd

with open('index.html', 'r', encoding='UTF8') as f:
    # step 01 : 데이터 수집
    contents = f.read()
    # step 02 : 데이터 파싱(순수한 HTML 파일을 BeautifulSoup 객체로 변환)
    soup = BeautifulSoup(contents, 'lxml')
    #print(soup)
    print(soup.h2)
    print(soup.ul)
    print("_____")
    print(soup.ul.li)

    # step 03 : 데이터 수집 위한 특정 태그 찾기
    companies = []
    print(soup.find_all('li'))

    # step 04 : 데이터 가공
    for tag in soup.find_all('li'):
        companies.append(tag.text)

    print(companies)

    # step 05 : 처리된 데이터 저장 pandas 데이터프레임
    crawling_dict = {'회사명' : companies}
    result = pd.DataFrame(crawling_dict)
    print(result)

    # step 06 : csv 파일로 내보내기 or DB로 내보내기 
    result.to_csv("result.csv", index=False)

<h2>모바일 컴퍼니</h2>
<ul id="mylist" style="width:150px">
<li>애플</li>
<li>삼성</li>
<li>노키아</li>
<li>LG</li>
</ul>
_____
<li>애플</li>
[<li>애플</li>, <li>삼성</li>, <li>노키아</li>, <li>LG</li>]
['애플', '삼성', '노키아', 'LG']
   회사명
0   애플
1   삼성
2  노키아
3   LG


In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

def crawling(soup) :
    # print(soup)
    tbody = soup.find("tbody")
    result = []
    for p in tbody.find_all('p', class_ = 'title'):
        result.append(p.get_text().strip())
    return result

def main() :
    custom_header = {
        'referer' : 'https://music.bugs.co.kr/',
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    url = "https://music.bugs.co.kr/chart" # 크롤링 하려는 웹사이트
    req = requests.get(url, headers = custom_header)
    
    soup = BeautifulSoup(req.text, "html.parser")

    titles = crawling(soup)
    print(pd.DataFrame({"노래제목" : titles}))

if __name__ == "__main__" :
    main()

                노래제목
0      Love wins all
1               Wife
2              To. X
3           Love 119
4      Perfect Night
..               ...
95             밤, 바다
96       Baggy Jeans
97        I Love You
98  그대가 있는 곳, 언제 어디든
99           사랑을 하다가

[100 rows x 1 columns]


In [64]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import time
import random

def crawling(soup, result):
    tbody = soup.find("table", class_="type2")
    
    for p in tbody.find_all('tr'):
        if len(p.get_text()) == 2:
            pass
        else:
            result.append(p.get_text().replace("\n", " ").replace("\t", ""))
    return result

def main():
    company_code = '005930' # 삼성전자
    url = "https://finance.naver.com/item/sise_day.nhn?code=" + company_code + "&page="
        
    headers = { 
                 'referer' : 'https://finance.naver.com/item/sise.naver?code=005930',
                 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
                }
    result = []
    for i in range(1, 13):
        req = requests.get(url + str(i), headers=headers)
        soup = BeautifulSoup(req.text, "html.parser")
       
        titles = crawling(soup, result)
    
    print(pd.DataFrame(titles))

if __name__ == "__main__":
    main()


                                                     0
0                              날짜 종가 전일비 시가 고가 저가 거래량 
1     2024.01.31 72,800   1,500   73,400 74,000 72,...
2     2024.01.30 74,300   100   75,000 75,300 73,70...
3     2024.01.29 74,400   1,000   73,800 75,200 73,...
4     2024.01.26 73,400   700   73,700 74,500 73,30...
..                                                 ...
127   2023.08.10 68,000   900   68,300 68,500 67,80...
128   2023.08.09 68,900   1,300   68,000 69,600 67,...
129   2023.08.08 67,600   900   69,000 69,100 67,40...
130   2023.08.07 68,500   200   67,700 69,200 67,60...
131   2023.08.04 68,300   500   68,800 69,100 68,20...

[132 rows x 1 columns]


In [89]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import time
import random

company_code = '005930' # 삼성전자
url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code
    
headers = { 
             'referer' : 'https://finance.naver.com/item/sise.naver?code=005930',
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
N = 2
#pd.concat() #진행
df = None
for pageNum in range(1, N+1):
    url = "https://finance.naver.com/item/sise_day.nhn?code=005930&page="+str(pageNum)
    # print(url)
    req = requests.get(url, headers=headers)
    # soup = BeautifulSoup(req.text, "html.parser")
    result =  pd.read_html(req.text, encoding='euc-kr')[0]
    #print(result)
    df = pd.concat([df, result], ignore_index = True)

df.dropna(inplace = True)
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.01.31,72700.0,1600.0,73400.0,74000.0,72500.0,13080752.0
2,2024.01.30,74300.0,100.0,75000.0,75300.0,73700.0,12244418.0
3,2024.01.29,74400.0,1000.0,73800.0,75200.0,73500.0,13976521.0
4,2024.01.26,73400.0,700.0,73700.0,74500.0,73300.0,11160062.0
5,2024.01.25,74100.0,100.0,74200.0,74800.0,73700.0,11737747.0
9,2024.01.24,74000.0,1200.0,75200.0,75200.0,73500.0,12860661.0
10,2024.01.23,75200.0,100.0,75700.0,75800.0,74300.0,14786224.0
11,2024.01.22,75100.0,400.0,75900.0,76000.0,75000.0,19673375.0
12,2024.01.19,74700.0,3000.0,73500.0,74700.0,73000.0,23363427.0
13,2024.01.18,71700.0,700.0,71600.0,72000.0,70700.0,17853397.0


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import time
import random
warnings.filterwarnings('ignore') 

def crawling(url, headers, soup):
    last_page = int(soup.select_one('td.pgRR').a['href'].split('=')[-1]) #모든 페이지 다 가져오기
    
    df = None
    count = 0
    for page in range(1, last_page + 1):
      req = requests.get(f'{url}&page={page}', headers=headers)
      df = pd.concat([df, pd.read_html(req.text, encoding = "euc-kr")[0]], ignore_index=True)
      if count > 10:
        break
      count += 1
      time.sleep( random.uniform(2,4)) 

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

def main():
    company_code = '005930' # 삼성전자
    url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code
    
    headers = { 
             'referer' : 'https://finance.naver.com/item/sise.naver?code=005930',
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    result = crawling(url, headers, soup)
    print(result)

if __name__ == "__main__":
    main()


             날짜       종가     전일비       시가       고가       저가         거래량
0    2024.02.01  73500.0   800.0  73000.0  74200.0  72900.0   9119409.0
1    2024.01.31  72700.0  1600.0  73400.0  74000.0  72500.0  15703560.0
2    2024.01.30  74300.0   100.0  75000.0  75300.0  73700.0  12244418.0
3    2024.01.29  74400.0  1000.0  73800.0  75200.0  73500.0  13976521.0
4    2024.01.26  73400.0   700.0  73700.0  74500.0  73300.0  11160062.0
..          ...      ...     ...      ...      ...      ...         ...
115  2023.08.11  67500.0   500.0  68400.0  68800.0  67500.0   9781038.0
116  2023.08.10  68000.0   900.0  68300.0  68500.0  67800.0  10227311.0
117  2023.08.09  68900.0  1300.0  68000.0  69600.0  67900.0  17259673.0
118  2023.08.08  67600.0   900.0  69000.0  69100.0  67400.0  14664709.0
119  2023.08.07  68500.0   200.0  67700.0  69200.0  67600.0  10968505.0

[120 rows x 7 columns]
